In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
data = pd.read_excel("副本工作簿1.xlsx", sheet_name="111 ")
data

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import linregress


# 函数：从Excel文件中提取期刊名称
def extract_journals_from_excel(file_path):
    try:
        df = pd.read_excel(file_path, sheet_name="111 ")  # 读取Excel文件
        if "SO" not in df.columns:
            print("Excel文件中未找到'SO'字段，请检查文件！")
            return []
        journals = df["SO"].dropna().tolist()  # 提取'SO'字段并去掉空值
        return journals
    except Exception as e:
        print(f"读取Excel文件时出错: {e}")
        return []


# 函数：分析布拉德福定律
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import linregress
from collections import Counter


def analyze_bradford(file_path):
    # 提取期刊数据（你已有的函数）
    journals = extract_journals_from_excel(file_path)
    if not journals:
        print("未找到期刊数据，请检查文件！")
        return

    # 统计每种期刊的文章数量
    journal_counts = Counter(journals)

    # 按文章数量降序排序
    sorted_journals = sorted(journal_counts.items(), key=lambda x: x[1], reverse=True)

    # 计算排名 n 和累积文章数 R(n)
    n = np.arange(1, len(sorted_journals) + 1)
    R_n = np.cumsum([count for _, count in sorted_journals])

    # 计算 ln(n)
    ln_n = np.log(n)

    # 线性回归
    slope, intercept, r_value, p_value, std_err = linregress(ln_n, R_n)
    regression_line = slope * ln_n + intercept

    # ---------------- 下面是可视化的改进示例 ----------------
    # 1. 使用 seaborn 风格
    sns.set_style("whitegrid")  # 也可选择 'darkgrid', 'white', 'ticks' 等

    # 2. 设置中文字体（如果你的系统支持 'SimHei'）
    plt.rcParams["font.sans-serif"] = ["SimHei"]
    plt.rcParams["axes.unicode_minus"] = False

    # 3. 创建图表
    fig, ax = plt.subplots(figsize=(9, 6), dpi=100)

    # 4. 绘制散点图
    #   - 调整散点大小 (s) 和透明度 (alpha) 来突出或弱化数据点
    ax.scatter(
        ln_n, R_n, color="blue", edgecolor="white", s=60, alpha=0.8, label="数据点"
    )

    # 5. 绘制线性拟合线
    #   - 调整线条宽度 (linewidth) 和样式 (linestyle)
    ax.plot(
        ln_n,
        regression_line,
        color="red",
        linewidth=2.0,
        linestyle="-",
        label=f"线性拟合: $R^2={r_value**2:.2f}$",
    )

    # 6. 标题与坐标轴
    #   - 可以考虑添加副标题或解释
    ax.set_title("布拉德福定律分析", fontsize=16, fontweight="bold", pad=15)
    ax.set_xlabel("ln(n)", fontsize=12, labelpad=10)
    ax.set_ylabel("R(n)", fontsize=12, labelpad=10)

    # 7. 网格线与背景
    #   - 已通过 seaborn 的 whitegrid 风格启用网格
    #   - 如果需要进一步定制网格线：
    ax.grid(True, linestyle="--", alpha=0.5)

    # 8. 美化坐标刻度
    #   - 可以根据数据范围选择合适的刻度步长
    ax.tick_params(axis="both", which="major", labelsize=11)

    # 9. 添加图例
    ax.legend(loc="upper left", frameon=True, fontsize=11)

    # 10. 使布局自适应，避免文字与边缘重叠
    plt.tight_layout()

    # 11. 显示图表
    plt.show()

    # ---------------- 分割线：回到分析逻辑的输出 ----------------
    print(f"线性回归的R²值: {r_value**2:.2f}")
    if r_value**2 > 0.9:
        print("数据符合布拉德福定律。")
        N = sum(journal_counts.values())
        target = N / 3  # 基于三区划分计算核心期刊
        core_journals = next(i for i, cumsum in enumerate(R_n, 1) if cumsum >= target)
        print(f"总文章数: {N}")
        print(f"核心期刊数量: {core_journals}")
    else:
        print("数据不符合布拉德福定律。可能是数据量不足或分布不均匀。")


# 使用示例
analyze_bradford("副本工作簿1.xlsx")

In [7]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import linregress


# 函数：分析齐普夫定律
def analyze_zipf(file_path):
    """分析期刊数据是否符合齐普夫定律，并计算参数 C"""
    # 提取期刊数据
    journals = extract_journals_from_excel(file_path)
    if not journals:
        print("未找到期刊数据，请检查文件！")
        return

    # 统计每种期刊的文章数量
    journal_counts = Counter(journals)

    # 按文章数量降序排序
    sorted_journals = sorted(journal_counts.items(), key=lambda x: x[1], reverse=True)

    # 提取排名 r 和文章数量 f(r)
    r = np.arange(1, len(sorted_journals) + 1)  # 排名从1开始
    f_r = np.array([count for _, count in sorted_journals])  # 文章数量

    # 计算 log(r) 和 log(f(r))
    log_r = np.log(r)
    log_f_r = np.log(f_r)

    # 进行线性回归
    slope, intercept, r_value, p_value, std_err = linregress(log_r, log_f_r)
    k = -slope  # 齐普夫定律中 k = -slope
    C = np.exp(intercept)  # C = e^{intercept}

    # 绘制 log(r) vs log(f(r)) 的散点图和拟合直线
    plt.figure(figsize=(8, 6))
    plt.scatter(log_r, log_f_r, color="blue", label="数据点")
    plt.plot(
        log_r,
        slope * log_r + intercept,
        color="red",
        label=f"线性拟合: R²={r_value**2:.2f}",
    )
    plt.xlabel("log(r)")
    plt.ylabel("log(f(r))")
    plt.title("齐普夫定律分析")
    plt.legend()
    plt.show()

    # 输出分析结果
    print(f"线性回归的R²值: {r_value**2:.2f}")
    print(f"参数 k: {k:.2f}")
    print(f"参数 C: {C:.2f}")

    # 判断是否符合齐普夫定律
    if r_value**2 > 0.9:
        print("数据符合齐普夫定律。")
    else:
        print("数据不符合齐普夫定律。可能是数据量不足或分布不均匀。")


# 使用示例：分析名为 'journal_data.xlsx' 的Excel文件
analyze_zipf("副本工作簿1.xlsx")

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

# 读取 Excel 文件，注意工作表名称和字段名
file_path = "副本工作簿1.xlsx"
sheet_name = "111 "

try:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
except Exception as e:
    print("读取文件出错:", e)
    raise

# 检查是否存在 'SO' 字段
if "SO" not in df.columns:
    raise ValueError("数据中没有找到 'SO' 字段，请检查数据。")

# 统计每个期刊的文章数量
journal_counts = df["SO"].value_counts()

# 构造频次分布：发表x篇文章的期刊数量
freq_distribution = journal_counts.value_counts().sort_index()

# 如果频次分布数据点少于2个，无法进行回归分析
if len(freq_distribution) < 2:
    raise ValueError("数据点不足，无法进行回归分析。")

# 准备对数转换数据
x = np.array(freq_distribution.index, dtype=float)
y = np.array(freq_distribution.values, dtype=float)

log_x = np.log(x)
log_y = np.log(y)

# 线性回归拟合： log(y) = log(C) - n*log(x)
slope, intercept, r_value, p_value, std_err = linregress(log_x, log_y)

C_value = np.exp(intercept)

# 绘制对数-对数图
plt.figure(figsize=(8, 6))
plt.scatter(log_x, log_y, color="blue", label="实际数据")
plt.plot(
    log_x,
    intercept + slope * log_x,
    "r-",
    label=f"拟合直线: slope={slope:.2f}, intercept={intercept:.2f}",
)
plt.xlabel("log(文章数 x)")
plt.ylabel("log(期刊数量 y)")
plt.title("期刊数据对数-对数图及洛特卡定律拟合")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("lotka_fit.png")
plt.show()

print("通过图像法验证，期刊数据近似符合洛特卡定律。")
print("参数 C 的值为：", C_value)
print("拟合直线的斜率（-n）为：", slope)
print("拟合优度 R² 为：", r_value**2)

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LinearRegression

# 读取Excel文件（请确保文件路径正确）
file_path = "副本工作簿1.xlsx"
df = pd.read_excel(file_path, sheet_name="111 ")

# 提取作者字段（假设作者列名为 'AU'，请根据实际列名调整）
authors = df["AU"].dropna().tolist()

# 统计每个作者的发文量
author_counts = defaultdict(int)
for entry in authors:
    # 拆分多个作者（假设用分号分隔）
    for author in entry.split(";"):
        author = author.strip()
        author_counts[author] += 1

# 生成发文量频次表
x_values = []
y_values = []
for count in author_counts.values():
    x_values.append(count)

# 统计每个x对应的Y（作者数量）
freq = defaultdict(int)
for x in x_values:
    freq[x] += 1

# 提取x和Y
x = np.array(list(freq.keys()))
Y = np.array(list(freq.values()))

# 对数转换（过滤x=0以避免log(0)）
valid_indices = (x > 0) & (Y > 0)
x_log = np.log(x[valid_indices]).reshape(-1, 1)
Y_log = np.log(Y[valid_indices])

# 线性回归
model = LinearRegression().fit(x_log, Y_log)
slope = model.coef_[0]
intercept = model.intercept_
C = np.exp(intercept)

# 绘制双对数散点图
plt.figure(figsize=(10, 6))
plt.scatter(x_log, Y_log, color="blue", label="Data Points")
plt.plot(
    x_log,
    model.predict(x_log),
    color="red",
    label=f"Fit: y = {slope:.2f}x + {intercept:.2f}",
)
plt.xlabel("ln(x) (发文量)")
plt.ylabel("ln(Y) (作者数量)")
plt.title("洛特卡定律验证（双对数坐标）")
plt.legend()
plt.grid(True)
plt.show()

# 输出结果
print(f"回归斜率: {slope:.4f}（应接近 -2）")
print(f"回归截距: {intercept:.4f}")
print(f"参数 C = e^{intercept:.4f} ≈ {C:.2f}")

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.linear_model import LinearRegression

# 读取Excel文件（请确保文件路径正确）
file_path = "副本工作簿1.xlsx"
df = pd.read_excel(file_path, sheet_name="111 ")

# 提取期刊名称字段（假设列名为 'SO'，请根据实际调整）
journals = df["SO"].dropna().tolist()

# 统计每个期刊的发文量
journal_counts = defaultdict(int)
for journal in journals:
    journal = journal.strip()
    journal_counts[journal] += 1

# 按发文量降序排序并分配排名
sorted_journals = sorted(journal_counts.items(), key=lambda x: -x[1])
ranks = np.arange(1, len(sorted_journals) + 1)
counts = np.array([count for _, count in sorted_journals])

# 对数转换（过滤0值）
valid_mask = (counts > 0) & (ranks > 0)
log_ranks = np.log(ranks[valid_mask]).reshape(-1, 1)
log_counts = np.log(counts[valid_mask])

# 线性回归
model = LinearRegression().fit(log_ranks, log_counts)
slope = model.coef_[0]
intercept = model.intercept_
C = np.exp(intercept)

# 绘制双对数散点图
plt.figure(figsize=(10, 6))
plt.scatter(log_ranks, log_counts, color="blue", label="Data Points")
plt.plot(
    log_ranks,
    model.predict(log_ranks),
    color="red",
    label=f"Fit: y = {slope:.2f}x + {intercept:.2f}",
)
plt.xlabel("ln(排名 r)")
plt.ylabel("ln(发文量 x)")
plt.title("齐普夫定律验证（双对数坐标）")
plt.legend()
plt.grid(True)
plt.show()

# 输出结果
print(f"回归斜率: {slope:.4f}（应接近 -1）")
print(f"回归截距: {intercept:.4f}")
print(f"参数 C = e^{intercept:.4f} ≈ {C:.2f}")

In [21]:
len(data["CR"].head(1).values[0].split(";"))
# 计算引文率 = 引用次数 / 文章总数
cited_list = []
for citations in data["CR"]:
    if pd.notnull(citations):
        cited_list.extend(citations.split(";"))
cited_count = len(set(cited_list))
citation_rate = cited_count / len(data)
print(f"引文率: {citation_rate:.2f}")

In [41]:
"""被引量最高的5本期刊中的：
期刊字段是JA，引文字段是CR,参考上面
（1）期刊被引量

（2）平均被引率

（3）2020年的影响因子

（4）2020年的当年指标"""
import pandas as pd
from collections import Counter
import numpy as np


# 解析CR字段，提取被引年份和期刊
def parse_cr(cr_str):
    # Check if cr_str is NaN or not a string
    if pd.isna(cr_str) or not isinstance(cr_str, str):
        return []  # Return empty list for invalid input
    # Proceed with splitting if it's a valid string
    citations = cr_str.split(";")
    result = []
    for citation in citations:
        parts = [part.strip() for part in citation.split(",")]
        if len(parts) >= 3:
            try:
                year = int(parts[1])  # Convert year to integer
                journal = parts[2]  # Extract journal name
                result.append((year, journal))
            except ValueError:
                continue  # Skip if year conversion fails
    return result


# 统计被引量
cited_journal_counter = Counter()
for index, row in data.iterrows():
    citations = parse_cr(row["CR"])
    for cited_year, cited_journal in citations:
        cited_journal_counter[cited_journal] += 1

# 确定Top 5期刊
top_5_journals = [journal for journal, count in cited_journal_counter.most_common(5)]

# 计算发表文章数
publication_counts = data.groupby(["SO", "PY"]).size().unstack(fill_value=0)
# 统计2020年文章对2018、2019、2020年文章的引用
citations_2020_to_2018 = Counter()
citations_2020_to_2019 = Counter()
citations_2020_to_2020 = Counter()

for index, row in data.iterrows():
    article_year = row["PY"]
    citations = parse_cr(row["CR"])
    if article_year == 2022:
        for cited_year, cited_journal in citations:
            if cited_year == 2020:
                citations_2020_to_2018[cited_journal] += 1
            elif cited_year == 2021:
                citations_2020_to_2019[cited_journal] += 1
            elif cited_year == 2022:
                citations_2020_to_2020[cited_journal] += 1
print(citations_2020_to_2018)
print(citations_2020_to_2019)
print(citations_2020_to_2020)
# 计算并输出指标
for journal in top_5_journals:
    print(journal)
    cited_count = cited_journal_counter[journal]
    for i in publication_counts.index:
        print(i)
        if journal in i:
            pub_total = publication_counts.loc[i, 2022]
            print(pub_total)
    print(f"Publication Total: {pub_total}, Cited Count: {cited_count}")
    # 平均被引率
    if pub_total > 0:
        avg_citation_rate = cited_count / pub_total
    else:
        avg_citation_rate = np.nan

    # 2020年影响因子
    if journal in publication_counts.index:
        pub_2018_2019 = (
            publication_counts.loc[journal, 2020]
            + publication_counts.loc[journal, 2021]
        )
        if pub_2018_2019 > 0:
            impact_factor_2020 = (
                citations_2020_to_2018[journal] + citations_2020_to_2019[journal]
            ) / pub_2018_2019
        else:
            impact_factor_2020 = np.nan
    else:
        impact_factor_2020 = np.nan

    # 2020年当年指标
    if journal in publication_counts.index and 2022 in publication_counts.columns:
        pub_2020 = publication_counts.loc[journal, 2022]
        print(pub_2020)
        if pub_2020 > 0:
            immediacy_index_2020 = citations_2020_to_2020[journal] / pub_2020
        else:
            immediacy_index_2020 = np.nan
    else:
        immediacy_index_2020 = 0

    # 使用print输出结果
    print(f"期刊: {journal}")
    print(f"  被引量: {cited_count}")
    print(f"  平均被引率: {avg_citation_rate:.2f}")
    print(f"  2020年影响因子: {impact_factor_2020:.2f}")
    print(f"  2020年当年指标: {immediacy_index_2020:.2f}")
    print()

In [7]:
# 引文量字段是NR，期刊字段是SO
# 总引文量分布如何
import matplotlib.pyplot as plt

data["NR"].describe()
# 总引文量分布（规律）如何？如何分析这一分布？

data["NR"].plot(
    kind="hist", bins=20, color="skyblue", edgecolor="black", figsize=(10, 6)
)
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False
plt.title("总引文量分布直方图")
plt.xlabel("总引文量")
plt.ylabel("频数")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [8]:
# 找出引文最多几个的期刊
top_journals = data.groupby("SO")["NR"].sum().nlargest(5)
top_journals